# Word Sense Disambiguation using Neural Networks
Adam Ek

----------------------------------

The lab is an exploration and learning exercise to be done in a group and also in discussion with the teachers and other students.

Before starting, please read the instructions on [how to work on group assignments](https://github.com/sdobnik/computational-semantics/blob/master/README.md).

Write all your answers and the code in the appropriate boxes below.

----------------------------------

A problem with static distributional vectors is the difficulty of distinguishing between different *word senses*. We will continue our exploration of word vectors by considering *trainable vectors* or *word embeddings* for Word Sense Disambiguation (WSD).

The goal of word sense disambiguation is to train a model to find the sense of a word (homonyms of a word-form). For example, the word "bank" can mean "sloping land" or "financial institution". 

(a) "I deposited my money in the **bank**" (financial institution)

(b) "I swam from the river **bank**" (sloping land)

In case a) and b) we can determine that the meaning of "bank" based on the *context*. To utilize context in a semantic model we use *contextualized word representations*. Previously we worked with *static word representations*, i.e. the representation does not depend on the context. To illustrate we can consider sentences (a) and (b), the word **bank** would have the same static representation in both sentences, which means that it becomes difficult for us to predict its sense. What we want is to create representations that depend on the context, i.e. *contextualized embeddings*. 

We will create contextualized embeddings with Recurrent Neural Networks. You can read more about recurrent neural netoworks [here](https://colah.github.io/posts/2015-08-Understanding-LSTMs/). Your overall task in this lab is to create a neural network model that can disambiguate the word sense of 30 different words. 

In [1]:
# first we import some packages that we need
import math
from collections import namedtuple
from pprint import pprint
import random
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import optim
from torch.nn import CrossEntropyLoss
import numpy as np
from sklearn.metrics import accuracy_score
import sys
from transformers import BertModel, BertTokenizer
import dill

# our hyperparameters (add more when/if you need them)
device = torch.device('cuda:3')

PADDING_TOKEN = '<PAD>'
UNKNOWN_TOKEN = '<UNK>'
UNKNOWN_LABEL = '<UNK>'
TRAINING_SPLIT = 0.8

2022-05-27 14:05:48.754893: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
torch.cuda.is_available()

True

# 1. Working with data

A central part of any machine learning system is the data we're working with. In this section we will split the data (the dataset is located here: ``wsd-data/wsd_data.txt``) into a training set and a test set. We will also create a baseline to compare our model against. Finally, we will use TorchText to transform our data (raw text) into a convenient format that our neural network can work with.

## Data

The dataset we will use contain different word sense for 30 different words. The data is organized as follows (values separated by tabs): 
- Column 1: word-sense
- Column 2: word-form
- Column 3: index of word
- Column 4: white-space tokenized context

### Splitting the data

Your first task is to seperate the data into a *training set* and a *test set*. The training set should contain 80% of the examples and the test set the remaining 20%. The examples for the test/training set should be selected **randomly**. Save each dataset into a .csv file for loading later. **[2 marks]**

In [3]:
def data_split(path_to_dataset):
    Sample = namedtuple('Sample', 'lemma word_sense word_form index context')

    samples = []
    with open(path_to_dataset, 'r') as f:
        for line in f:
            word_sense, word_form, index, context = line.split('\t')
            word, form = word_form.split('.')
            samples.append(Sample(word.lower(), word_sense, form, int(index),
                                  [context_word.lower() for context_word in context.split(' ')]))

    random.shuffle(samples)
    split = math.floor(len(samples) * TRAINING_SPLIT)
    return samples[:split], samples[split:]

In [4]:
train, test = data_split('wsd_data_small.txt')
print(len(train), len(test))

# for sample in train[:10]:
#    print(sample.word, sample.word_sense, sample.word_form)

24 6


### Creating a baseline

Your second task is to create a *baseline* for the task. A baseline is a "reality check" for a model, given a very simple heuristic/algorithmic/model solution to the problem, can our neural network perform better than this?
The baseline you are to create is the "most common sense" (MCS) baseline. For each word form, find the most commonly assigned sense to the word, and label a words with that sense. **[2 marks]**

E.g. In a fictional dataset, "bank" have two senses, "financial institution" which occur 5 times and "side of river" 3 times. Thus, all 8 occurences of bank is labeled "financial institution" and this yields an MCS accuracy of 5/8 = 62.5%. If a model obtain a higher score than this, we can conclude that the model *at least* is better than selecting the most frequent word sense.

In [5]:
def mcs_baseline(data):
    word_senses = {}
    for sample in data:
        word, sense, *_ = sample

        if word not in word_senses:
            word_senses[word] = {}

        word_senses[word][sense] = word_senses.get(word, {}).get(sense, 0) + 1

    return {w: max(s, key=s.get) for w, s in word_senses.items()}

In [6]:
baseline = mcs_baseline(train)
print(len(baseline))
pprint(baseline)

15
{'bad': 'bad%3:00:00::',
 'build': 'build%2:36:00::',
 'follow': 'follow%2:42:02::',
 'force': 'force%1:07:01::',
 'hold': 'hold%2:31:01::',
 'line': 'line%1:04:01::',
 'national': 'national%5:00:00:public:00',
 'physical': 'physical%3:00:00::',
 'place': 'place%1:04:00::',
 'position': 'position%1:04:01::',
 'professional': 'professional%3:01:01::',
 'regular': 'regular%5:00:00:standard:02',
 'see': 'see%2:31:00::',
 'serve': 'serve%2:41:02::',
 'time': 'time%1:28:00::'}


### Creating data iterators

To train a neural network, we first need to prepare the data. This involves converting words (and labels) to a number, and organizing the data into batches. We also want the ability to shuffle the examples such that they appear in a random order.  

To do all of this we will use the torchtext library (https://torchtext.readthedocs.io/en/latest/index.html). In addition to converting our data into numerical form and creating batches, it will generate a word and label vocabulary, and data iterators than can sort and shuffle the examples. 

Your task is to create a dataloader for the training and test set you created previously. So, how do we go about doing this?

1) First we create a ``Field`` for each of our columns. A field is a function which tokenize the input, keep a dictionary of word-to-numbers, and fix paddings. So, we need four fields, one for the word-sense, one for the position, one for the lemma and one for the context. 

2) After we have our fields, we need to process the data. For this we use the ``TabularDataset`` class. We pass the name and path of the training and test files we created previously, then we assign which field to use in each column. The result is that each column will be processed by the field indicated. So, the context column will be tokenized and processed by the context field and so on. 

3) After we have processed the dataset we need to build the vocabulary, for this we call the function ``build_vocab()`` on the different ``Fields`` with the output from ``TabularDataset`` as input. This looks at our dataset and creates the necessary vocabularies (word-to-number mappings). 

4) Finally, the last step. In the last step we load the data objects given by the ``TabularDataset`` and pass it to the ``BucketIterator`` class. This class will organize our examples into batches and shuffle them around (such that for each epoch the model observe the examples in a different order). When we are done with this we can let our function return the data iterators and vocabularies, then we are ready to train and test our model!

Implement the dataloader. [**2 marks**]

*hint: for TabularDataset and BucketIterator use the class function splits()*

In [7]:
class WordSenseDataset(Dataset):
    def __init__(self, data, dataset=None):
        self.max_length_context = -1
        if dataset is None:
            vocab = {PADDING_TOKEN, UNKNOWN_TOKEN}
            senses = {UNKNOWN_LABEL}
            for sample in data:
                vocab.update(sample.context)
                senses.add(sample.word_sense)
                self.max_length_context = max(self.max_length_context, len(sample.context))

            self.vocab = {word: index for index, word in enumerate(list(vocab))}
            self.senses = {sense: index for index, sense in enumerate(list(senses))}
        else:
            self.vocab = dataset.vocab
            self.senses = dataset.senses
            self.max_length_context = dataset.max_length_context

        self.samples = []
        for sample in data:
            padded_context = [self.get_encoded_word(word) for word in sample.context]
            padded_context.extend([self.get_encoded_word(PADDING_TOKEN)] * (self.max_length_context - len(sample.context)))
            self.samples.append((
                sample.lemma,
                self.get_encoded_label(sample.word_sense),
                sample.index,
                torch.tensor(padded_context)
            ))

    def __getitem__(self, idx):
        Sample = namedtuple('Sample', 'lemma word_sense position context')

        return Sample(*self.samples[idx])

    def __len__(self):
        return len(self.samples)

    def get_encoded_word(self, word):
        if word in self.vocab:
            return self.vocab[word]
        else:
            return self.vocab[UNKNOWN_TOKEN]

    def get_encoded_label(self, label):
        if label in self.senses:
            return self.senses[label]
        else:
            return self.senses[UNKNOWN_LABEL]

    def get_vocab_size(self):
        return len(self.vocab)

In [8]:
# word_sense_dataset = WordSenseDataset(train)
# print(word_sense_dataset[0])

In [9]:
# dataloader = DataLoader(word_sense_dataset,
#                         batch_size=batch_size,
#                         shuffle=True)

In [10]:
def dataloader(path, batch_size):
    train, test = data_split(path)

    train_dataset = WordSenseDataset(train)
    test_dataset = WordSenseDataset(test, train_dataset)

    train_dataloader = DataLoader(train_dataset,
                                  batch_size=batch_size,
                                  shuffle=True)
    test_dataloader = DataLoader(test_dataset,
                                 batch_size=1,
                                 shuffle=True)

    return train_dataloader, test_dataloader


# 2.1 Creating and running a Neural Network for WSD

In this section we will create and run a neural network to predict word senses based on *contextualized representations*.

### Model

We will use a bidirectional Long-Short-Term Memory (LSTM) network to create a representation for the sentences and a Linear classifier to predict the sense of each word.

When we initialize the model, we need a few things:

    1) An embedding layer: a dictionary from which we can obtain word embeddings
    2) A LSTM-module to obtain contextual representations
    3) A classifier that compute scores for each word-sense given *some* input


The general procedure is the following:

    1) For each word in the sentence, obtain word embeddings
    2) Run the embedded sentences through the RNN
    3) Select the appropriate hidden state
    4) Predict the word-sense 

**Suggestion for efficiency:**  *Use a low dimensionality (32) for word embeddings and the LSTM when developing and testing the code, then scale up when running the full training/tests*
    
Your tasks will be to create two different models (both follow the two outlines described above), described below:

In [11]:
hyperparameters = {
    'embedding_dim': 64,
    'out_dim': 64,
    'epochs': 3,
    'learning_rate': 0.001,
    'batch_size': 32
}

In the first approach to WSD, you are to select the index of our target word (column 3 in the dataset) and predict the word sense. **[5 marks]**


In [12]:
class WSDModel_approach1(nn.Module):
    def __init__(self, vocab_size, embedding_dim, out_dim, word_sense_size, padding_idx):
        super(WSDModel_approach1, self).__init__()

        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
        self.rnn = nn.LSTM(embedding_dim, out_dim, num_layers=1, bidirectional=True, batch_first=True)
        self.classifier = nn.Linear(out_dim * 2, word_sense_size)

    def forward(self, context_batch, position_batch):
        embeddings = self.embeddings(context_batch)
        output, _ = self.rnn(embeddings)
        positioned_output = output[torch.arange(output.size(0)), position_batch]
        predictions = self.classifier(positioned_output)

        return predictions

In the second approach to WSD, you are to predict the word sense based on the final hidden state given by the RNN. **[5 marks]**

In [13]:
class WSDModel_approach2(nn.Module):
    def __init__(self, vocab_size, embedding_dim, out_dim, word_sense_size, padding_idx):
        super(WSDModel_approach2, self).__init__()

        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
        self.rnn = nn.LSTM(embedding_dim, out_dim, num_layers=1, bidirectional=True, batch_first=True)
        self.classifier = nn.Linear(out_dim * 2, word_sense_size)

    def forward(self, context_batch):
        embeddings = self.embeddings(context_batch)
        _, (h_n, _) = self.rnn(embeddings)
        predictions = self.classifier(torch.cat((h_n[0,:,:], h_n[1,:,:]), dim=1))

        return predictions

### Training and testing the model

Now we are ready to train and test our model. What we need now is a loss function, an optimizer, and our data. 

- First, create the loss function and the optimizer.
- Next, we iterate over the number of epochs (i.e. how many times we let the model see our data). 
- For each epoch, iterate over the dataset (``train_iter``) to obtain batches. Use the batch as input to the model, and let the model output scores for the different word senses.
- For each model output, calculate the loss (and print the loss) on the output and update the model parameters.
- Reset the gradients and repeat.
- After all epochs are done, test your trained model on the test set (``test_iter``) and calculate the total and per-word-form accuracy of your model.

Implement the training and testing of the model **[4 marks]**

**Suggestion for efficiency:** *when developing your model, try training and testing the model on one or two batches (for each epoch) of data to make sure everything works! It's very annoying if you train for N epochs to find out that something went wrong when testing the model, or to find that something goes wrong when moving from epoch 0 to epoch 1.*

In [14]:
train_dataloader, test_dataloader = dataloader('wsd_data_small.txt', hyperparameters['batch_size'])
vocab = train_dataloader.dataset.vocab
labels = train_dataloader.dataset.senses

loss_function = CrossEntropyLoss()

In [15]:
with open('dataloaders.dill', 'wb') as f:
    dill.dump((train_dataloader, test_dataloader), f)

In [16]:
wsd1_model = WSDModel_approach1(len(vocab),
                                hyperparameters['embedding_dim'],
                                hyperparameters['out_dim'],
                                len(labels),
                                vocab[PADDING_TOKEN])
wsd1_model.to(device)

wsd1_optimizer = optim.Adam(wsd1_model.parameters(), lr=hyperparameters['learning_rate'])

In [17]:
print(
    f'{hyperparameters["epochs"]} EPOCHS - {math.floor(len(train_dataloader.dataset) / train_dataloader.batch_size)} BATCHES PER EPOCH')

for epoch in range(hyperparameters['epochs']):
    total_loss = 0
    for i, batch in enumerate(train_dataloader):
        batch_senses = batch.word_sense.to(device)
        batch_contexts = batch.context.to(device)
        batch_indices = batch.position.to(device)

        output = wsd1_model(batch_contexts, batch_indices)

        loss = loss_function(output.view(-1, len(labels)), batch_senses.view(-1))
        total_loss += loss.item()

        # print average loss for the epoch
        sys.stdout.write(f'\repoch {epoch}, batch {i}: {np.round(total_loss / (i + 1), 4)}')

        # compute gradients
        loss.backward()

        # update parameters
        wsd1_optimizer.step()

        # reset gradients
        wsd1_optimizer.zero_grad()
    print()

3 EPOCHS - 0 BATCHES PER EPOCH


RuntimeError: cuDNN error: CUDNN_STATUS_INTERNAL_ERROR

In [ ]:
wsd1_gold_labels = []
wsd1_predicted_labels = []

for sample in test_dataloader:
    context = sample.context.to(device)
    position = sample.position.to(device)
    predicted_label = wsd1_model(context, position).cpu().detach()
    max_label = torch.max(predicted_label[0], dim=0)
    wsd1_gold_labels.append(int(max_label.indices))
    wsd1_predicted_labels.append(int(sample.word_sense[0]))

In [ ]:
print(accuracy_score(wsd1_gold_labels, wsd1_predicted_labels))

In [ ]:
with open('wsd1.dill', 'wb') as f:
    dill.dump(wsd1_model, f)

In [ ]:
pprint(hyperparameters)
print('vocab_size:', len(vocab))
print('label_size:', len(labels))
print('len_context:', len(train_dataloader.dataset[0].context))

In [ ]:
wsd2_model = WSDModel_approach2(len(vocab),
                                hyperparameters['embedding_dim'],
                                hyperparameters['out_dim'],
                                len(labels),
                                vocab[PADDING_TOKEN])
wsd2_model.to(device)

wsd2_optimizer = optim.Adam(wsd2_model.parameters(), lr=hyperparameters['learning_rate'])

In [ ]:
print(
    f'{hyperparameters["epochs"]} EPOCHS - {math.floor(len(train_dataloader.dataset) / train_dataloader.batch_size)} BATCHES PER EPOCH')

for epoch in range(hyperparameters['epochs']):
    total_loss = 0
    for i, batch in enumerate(train_dataloader):
        batch_senses = batch.word_sense.to(device)
        batch_contexts = batch.context.to(device)

        output = wsd2_model(batch_contexts)
        loss = loss_function(output.view(-1, len(labels)), batch_senses.view(-1))
        total_loss += loss.item()

        # print average loss for the epoch
        sys.stdout.write(f'\repoch {epoch}, batch {i}: {np.round(total_loss / (i + 1), 4)}')

        # compute gradients
        loss.backward()

        # update parameters
        wsd2_optimizer.step()

        # reset gradients
        wsd2_optimizer.zero_grad()
    print()

In [ ]:
wsd2_gold_labels = []
wsd2_predicted_labels = []

for sample in test_dataloader:
    context = sample.context.to(device)
    predicted_label = wsd2_model(context).cpu().detach()
    max_label = torch.max(predicted_label[0], dim=0)
    wsd2_gold_labels.append(int(max_label.indices))
    wsd2_predicted_labels.append(int(sample.word_sense[0]))

In [ ]:
print(accuracy_score(wsd2_gold_labels, wsd2_predicted_labels))

In [ ]:
with open('wsd2.dill', 'wb') as f:
    dill.dump(wsd2_model, f)

In [ ]:
torch.save(wsd1_model.state_dict(), 'wsd1.pt')
torch.save(wsd2_model.state_dict(), 'wsd2.pt')

# 2.2 Running a transformer for WSD

In this section of the lab you'll try out the transformer, specifically the BERT model. For this we'll use the huggingface library (https://huggingface.co/).

You can find the documentation for the BERT model here (https://huggingface.co/transformers/model_doc/bert.html) and a general usage guide here (https://huggingface.co/transformers/quickstart.html).

What we're going to do is *fine-tune* the BERT model, i.e. update the weights of a pre-trained model. That is, we have a model that is trained on language modeling, but now we apply it to word sense disambiguation with the word representations it learnt from language modeling.

We'll use the same data splits for training and testing as before, but this time you'll not use a torchtext dataloader. Rather now you create an iterator that collects N sentences (where N is the batch size) then use the BertTokenizer to transform the sentence into integers. For your dataloader, remember to:
* Shuffle the data in each batch
* Make sure you get a new iterator for each *epoch*
* Create a vocabulary of *sense-labels* so you can calculate accuracy 

We then pass this batch into the BERT model and train as before. The BERT model will encode the sentence, then we send this encoded sentence into a prediction layer (you can either the the sentence-representation from bert, or the ambiguous word) like before and collect sense predictions.

About the hyperparameters and training:
* For BERT, usually a lower learning rate works best, between 0.0001-0.000001.
* BERT takes alot of resources, running it on CPU will take ages, utilize the GPUs :)
* Since BERT takes alot of resources, use a small batch size (4-8)
* Computing the BERT representation, make sure you pass the mask

**[10 marks]**

In [ ]:
bert_hyperparameters = {
    'epochs': 1,
    'learning_rate': 0.00001,
    'batch_size': 16
}

In [ ]:
def bert_data_split(path_to_dataset):
    Sample = namedtuple('Sample', 'word_sense context')

    samples = []
    with open(path_to_dataset, 'r') as f:
        for line in f:
            word_sense, word_form, _, context = line.split('\t')
            lemma, _ = word_form.split('.')
            samples.append(Sample(lemma, context))

    random.shuffle(samples)
    split = math.floor(len(samples) * TRAINING_SPLIT)
    return samples[:split], samples[split:]

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, data, bert_tokenizer, dataset=None):
        if dataset is None:
            senses = {UNKNOWN_LABEL}
            for sample in data:
                senses.add(sample.word_sense)
            self.senses = {sense: index for index, sense in enumerate(list(senses))}
        else:
            self.senses = dataset.senses
            
        contexts = bert_tokenizer([sample.context for sample in data],
                                  padding=True,
                                  truncation=True,
                                  return_tensors='pt')
        self.samples = []
        for index, sample in enumerate(data):
            self.samples.append((
                self.get_encoded_label(sample.word_sense),
                {k: v[index] for k, v in contexts.items()}  
            ))

    def __getitem__(self, idx):
        Sample = namedtuple('Sample', 'word_sense context')

        return Sample(*self.samples[idx])

    def __len__(self):
        return len(self.samples)
    
    def get_encoded_label(self, label):
        if label in self.senses:
            return self.senses[label]
        else:
            return self.senses[UNKNOWN_LABEL]

In [ ]:
def BERT_dataloader(path, batch_size, bert_tokenizer):
    train, test = bert_data_split(path)

    train_dataset = BERTDataset(train, bert_tokenizer)
    test_dataset = BERTDataset(test, bert_tokenizer, dataset=train_dataset)

    train_dataloader = DataLoader(train_dataset,
                                  batch_size=batch_size,
                                  shuffle=True)
    test_dataloader = DataLoader(test_dataset,
                                 batch_size=1,
                                 shuffle=True)

    return train_dataloader, test_dataloader

In [ ]:
class BERT_WSD(nn.Module):
    def __init__(self, model, label_size):
        super(BERT_WSD, self).__init__()
        # your code goes here
        self.bert = model
        self.classifier = nn.Linear(768, label_size)

    def forward(self, batch):
        # your code goes here
        bert = self.bert(**batch)
        predictions = self.classifier(bert.pooler_output)

        return predictions

In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
bert_train_dataloader, bert_test_dataloader = BERT_dataloader('wsd_data_medium.txt', bert_hyperparameters['batch_size'], bert_tokenizer)
labels = bert_train_dataloader.dataset.senses

In [ ]:
loss_function = CrossEntropyLoss()

bert_model = BERT_WSD(BertModel.from_pretrained("bert-base-uncased"), len(labels))
bert_model.to(device)
optimizer = optim.Adam(bert_model.parameters(), lr=bert_hyperparameters['learning_rate'])

In [ ]:
print(
    f'{bert_hyperparameters["epochs"]} EPOCHS - {math.floor(len(bert_train_dataloader.dataset) / bert_train_dataloader.batch_size)} BATCHES PER EPOCH')
for epoch in range(bert_hyperparameters['epochs']):
    total_loss = 0
    for i, batch in enumerate(bert_train_dataloader):
        context_batch = {k: v.to(device) for k, v in batch.context.items()}
        
        labels = batch.word_sense.to(device)

        output = bert_model(context_batch)
        
        loss = loss_function(output, labels)
        total_loss += loss.item()

        # print average loss for the epoch
        sys.stdout.write(f'\repoch {epoch}, batch {i}: {np.round(total_loss / (i + 1), 4)}')

        # compute gradients
        loss.backward()

        # update parameters
        optimizer.step()

        # reset gradients
        optimizer.zero_grad()
    print()
# test model after all epochs are completed

In [ ]:
bert_gold_labels = []
bert_predicted_labels = []

for sample in bert_test_dataloader:
    context = {k: v.to(device) for k, v in sample.context.items()}
    predicted_label = bert_model(context).cpu().detach()
    max_label = torch.max(predicted_label[0], dim=0)
    bert_predicted_labels.append(int(max_label.indices))
    bert_gold_labels.append(int(sample.word_sense[0]))

In [ ]:
print(accuracy_score(bert_gold_labels, bert_predicted_labels))

In [ ]:
with open('bert.dill', 'wb') as f:
    dill.dump(bert_model, f)

In [ ]:
torch.save(bert_model.state_dict(), 'bert.pt')

# 3. Evaluation

Explain the difference between the first and second approach. What kind of representations are the different approaches using to predict word-senses? **[4 marks]**

> The first approach uses the output at the timestep of the target word. This represents the target word in this specific context and hopefully after training in a generic context.
>
> The second approach is using the final hidden state of the BiLSTM, which represents the whole sentence, in which the word is occuring. As expected this results in a worse accuracy, since it focuses not only on the target word, but encodes also the rest of context.

Evaluate your model with per-word-form *accuracy* and comment on the results you get, how does the model perform in comparison to the baseline, and how do the models compare to each other? 

Expand on the evaluation by sorting the word-forms by the number of senses they have. Are word-forms with fewer senses easier to predict? Give a short explanation of the results you get based on the number of senses per word.

**[6 marks]**

How does the LSTMs perform in comparison to BERT? What's the difference between representations obtained by the LSTMs and BERT? **[2 marks]**

What could we do to improve our LSTM word sense disambiguation models and our BERT model? **[4 marks]**

# Readings:

[1] Kågebäck, M., & Salomonsson, H. (2016). Word Sense Disambiguation using a Bidirectional LSTM. arXiv preprint arXiv:1606.03568.

[2] https://cl.lingfil.uu.se/~nivre/master/NLP-LexSem.pdf